In [18]:
import time
import requests
import pandas as pd
import json

In [52]:
url = "https://api.upbit.com/v1/candles/minutes/1"

In [53]:
params = {
    "market": "KRW-BTC",
    "count": "200"
}

In [54]:
response = requests.get(url, params=params)

In [55]:
response_dict = json.loads(response.text)

In [56]:
data_df = pd.DataFrame(response_dict)

In [57]:
data_df

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-BTC,2022-03-12T09:50:00,2022-03-12T18:50:00,48880000.0,48880000.0,48880000.0,48880000.0,1647078600319,1.275653e+06,0.026098,1
1,KRW-BTC,2022-03-12T09:49:00,2022-03-12T18:49:00,48872000.0,48881000.0,48864000.0,48880000.0,1647078599475,4.575457e+07,0.936193,1
2,KRW-BTC,2022-03-12T09:48:00,2022-03-12T18:48:00,48871000.0,48876000.0,48870000.0,48872000.0,1647078538462,5.828570e+07,1.192641,1
3,KRW-BTC,2022-03-12T09:47:00,2022-03-12T18:47:00,48870000.0,48876000.0,48869000.0,48870000.0,1647078479405,7.808485e+07,1.597760,1
4,KRW-BTC,2022-03-12T09:46:00,2022-03-12T18:46:00,48868000.0,48870000.0,48868000.0,48870000.0,1647078416286,9.869521e+07,2.019561,1
...,...,...,...,...,...,...,...,...,...,...,...
195,KRW-BTC,2022-03-12T06:35:00,2022-03-12T15:35:00,48728000.0,48745000.0,48701000.0,48702000.0,1647066959568,3.644340e+07,0.748154,1
196,KRW-BTC,2022-03-12T06:34:00,2022-03-12T15:34:00,48736000.0,48736000.0,48706000.0,48724000.0,1647066898830,4.283610e+07,0.879192,1
197,KRW-BTC,2022-03-12T06:33:00,2022-03-12T15:33:00,48734000.0,48760000.0,48714000.0,48714000.0,1647066835433,5.578092e+07,1.144558,1
198,KRW-BTC,2022-03-12T06:32:00,2022-03-12T15:32:00,48735000.0,48736000.0,48734000.0,48734000.0,1647066779228,1.009804e+08,2.072042,1


In [48]:
last_time = response_dict[-1]['candle_date_time_kst']

In [51]:
response_dict

{'error': {'name': 400,
  'message': 'Invalid parameter. Check the given value!'}}

In [50]:
for i in range(10):
    last_time = response_dict[-1]['candle_date_time_kst']
    print(last_time)
    params = {
        "market": "KRW-BTC",
        "count": "200",
        "to": last_time,
    }
    response = requests.get(url, params=params)
    response_dict = json.loads(response.text)
    data_df = pd.concat([data_df, pd.DataFrame(response_dict)])
    time.sleep(0.1)

2022-03-12T15:30:00


KeyError: -1